In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, ReLU, Concatenate, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.models import Model

# Define input shape
input_shape = (100, 100, 3)

# Define input layer
inputs = Input(shape=input_shape)

# Traditional convolutional layers
conv1 = Conv2D(64, (3, 3), padding='same')(inputs)
bn1 = BatchNormalization()(conv1)
relu1 = ReLU()(bn1)

conv2 = Conv2D(64, (5, 5), padding='same')(relu1)
bn2 = BatchNormalization()(conv2)
relu2 = ReLU()(bn2)

# Blocks of parallel convolutional layers
def block(inputs, filters):
    conv3 = Conv2D(filters, (3, 3), padding='same')(inputs)
    bn3 = BatchNormalization()(conv3)
    relu3 = ReLU()(bn3)
    
    conv4 = Conv2D(filters, (5, 5), padding='same')(inputs)
    bn4 = BatchNormalization()(conv4)
    relu4 = ReLU()(bn4)
    
    conv5 = Conv2D(filters, (7, 7), padding='same')(inputs)
    bn5 = BatchNormalization()(conv5)
    relu5 = ReLU()(bn5)
    
    conv6 = Conv2D(filters, (11, 11), padding='same')(inputs)
    bn6 = BatchNormalization()(conv6)
    relu6 = ReLU()(bn6)
    
    # Concatenate output of parallel conv layers
    concat = Concatenate()([relu3, relu4, relu5, relu6])
    
    return concat

# Block 1
block1 = block(relu2, 128)

# Block 2
block2 = block(block1, 256)

# Block 3
block3 = block(block2, 512)

# Block 4
conv7 = Conv2D(128, (1, 1), padding='same')(relu2)
bn7 = BatchNormalization()(conv7)
relu7 = ReLU()(bn7)

block4 = block(relu7, 256)

# Concatenate outputs
concat1 = Concatenate()([block2, block3])
concat2 = Concatenate()([block1, block4])

# Average pooling layer
pooling = MaxPooling2D((2, 2))(concat2)

# Flatten
flatten = Flatten()(pooling)

# Fully connected layers
fc1 = Dense(1024, activation='relu')(flatten)
dropout1 = Dropout(0.5)(fc1)

fc2 = Dense(512, activation='relu')(dropout1)
dropout2 = Dropout(0.5)(fc2)

output = Dense(36, activation='softmax')(dropout2)

# Define model
model = Model(inputs=inputs, outputs=output)

# Print the number of layers
print("Number of layers:", len(model.layers))

# Print model summary
model.summary()



/Users/menna/opt/anaconda3/envs/conda_env/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2023-05-04 01:22:33.065220: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Number of layers: 44
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 100, 100, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 100, 100, 64  1792        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 100, 100, 64  256        ['conv2d[0][0]']                 
 alization)                     )                                        

In [16]:
import os
import cv2
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split


# Define the path to your dataset directory
data_dir = "data/train/"

# Define the image size
img_size = (100, 100)

# Initialize empty lists to store images and labels
images = []
labels = []

# Loop through each subdirectory in the dataset directory
for subdir in os.listdir(data_dir):
    subdir_path = os.path.join(data_dir, subdir)
    # Loop through each image in the subdirectory
    for img_file in os.listdir(subdir_path):
        img_path = os.path.join(subdir_path, img_file)
        
        # Read the image and resize it to the desired size
        img = cv2.imread(img_path)
        # Convert the image to the sRGB color space
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        # Save the image in the sRGB color space
        cv2.imwrite(img_path, img)
        if img is not None and img.size > 0:
            # Resize the image
            img = cv2.resize(img, (100, 100))
        else:
            print("Error: Failed to load image or image is empty.  " + img_path)

        # Add the image and label to the lists
        images.append(img)
        labels.append(subdir)
        
# Convert the lists to numpy arrays
images = np.array(images)
labels = np.array(labels)

# Encode the labels using LabelEncoder
le = LabelEncoder()
labels = le.fit_transform(labels)

# Convert the labels to one-hot encoding
labels = to_categorical(labels)

# Normalize the images
images = images.astype('float32') / 255.0

# Split the dataset into training, validation, and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
# Print the shapes of the data
print("Training images shape:", X_train.shape)
print("Training labels shape:", y_train.shape)
print("Validation images shape:", X_val.shape)
print("Validation labels shape:", y_val.shape)
print("Testing images shape:", X_test.shape)
print("Testing labels shape:", y_test.shape)

libpng warning: iCCP: known incorrect sRGB profile


Training images shape: (2491, 100, 100, 3)
Training labels shape: (2491, 36)
Testing images shape: (623, 100, 100, 3)
Testing labels shape: (623, 36)


In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model on the training set
history = model.fit(X_train, y_train, 
                    epochs=100, 
                    batch_size=128, 
                    validation_data=(X_val, y_val))

# Evaluate the performance of the model on the testing set
test_loss, test_acc = model.evaluate(X_test, y_test)
print('Test accuracy:', test_acc)

# Save the trained model
model.save('model.h5')

In [13]:
import cv2

# Load the image
img = cv2.imread("data/train/bell pepper/Image_56.jpg")

# Check that the image is not empty
if img is not None and img.size > 0:
    # Resize the image
    resized_img = cv2.resize(img, (200, 50))
else:
    print("Error: Failed to load image or image is empty.")


Error: Failed to load image or image is empty.


In [3]:
import os

# Define the path to the dataset directory
data_dir = 'data'

# Loop over the files in the directory
for file_name in os.listdir(data_dir):
    # Check if the file is a .DS_Store file
    if file_name == '.DS_Store':
        # Delete the file
        os.remove(os.path.join(data_dir, file_name))
